In [1]:
!pip3 install pandas scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd

# Load the processed dataset
df = pd.read_csv("tacticone_dataset.csv")

# Features (TF-IDF values)
X = df.drop(columns=["query", "category", "processed_query"])  

# Target labels (query categories)
y = df["category"]

In [3]:
from sklearn.model_selection import train_test_split

# Split dataset (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
from sklearn.linear_model import LogisticRegression

# Initialize and train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [5]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on test data
y_pred = model.predict(X_test)

# Print evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 1.0

Classification Report:
                      precision    recall  f1-score   support

       order_status       1.00      1.00      1.00        52
product_information       1.00      1.00      1.00        57
            returns       1.00      1.00      1.00        46
  technical_support       1.00      1.00      1.00        45

           accuracy                           1.00       200
          macro avg       1.00      1.00      1.00       200
       weighted avg       1.00      1.00      1.00       200



In [6]:
import joblib

# Save the trained model
joblib.dump(model, "query_classification_model.pkl")


['query_classification_model.pkl']

In [7]:
import joblib

# Load the saved model
model = joblib.load("query_classification_model.pkl")


In [8]:
import string

# Define text preprocessing function
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation
    return text

# Example query
new_query = "Where is my order?"
processed_query = preprocess_text(new_query)


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Load the dataset to get the original TF-IDF structure
df = pd.read_csv("tacticone_dataset.csv")

# Recreate the TF-IDF vectorizer with original training data
vectorizer = TfidfVectorizer()
vectorizer.fit(df["processed_query"])  # Fit on full dataset

# Transform the new query
query_tfidf = vectorizer.transform([processed_query])


In [10]:
# Predict category
predicted_category = model.predict(query_tfidf)[0]
print("Predicted Category:", predicted_category)


Predicted Category: order_status


/Users/niranjanbhol/Documents/tacticone/venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [11]:
!pip3 install openai


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [12]:
!pip install python-dotenv  # Install dotenv (if not already installed)
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv("OPENAI_API_KEY")

# Verify key retrieval (Don't print the key directly in production)
print("API Key Loaded:", bool(openai_api_key))  # Should return True if key is loaded



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
API Key Loaded: True


In [13]:
import openai
import os
from dotenv import load_dotenv

# Load API Key from .env
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Set the API key
client = openai.OpenAI(api_key=openai_api_key)

# Function to generate AI response using the new OpenAI API format
def generate_ai_response(query):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful support assistant."},
            {"role": "user", "content": query}
        ]
    )
    return response.choices[0].message.content

# Test API call
print(generate_ai_response("How do I return my order?"))


To return your order, you may follow these general steps:

1. Review the return policy of the company from which you made the purchase. The return policy will typically provide important details on eligibility, timeframes, and requirements for returning items.

2. Check if your item is eligible for a return. Some items, such as personalized or used items, may not be returnable.

3. Prepare the item for return by ensuring it is in its original condition with all tags and packaging included.

4. Contact the company's customer service to initiate the return process. They may provide you with a return authorization number or label to include with the returned item.

5. Package the item securely and send it back to the company's specified return address. Make sure to use a shipping method that provides tracking information.

6. Once the company receives the returned item and verifies its condition, a refund or exchange may be processed according to their return policy.

Remember to keep rec

In [14]:
!pip install fastapi uvicorn nest_asyncio


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import uvicorn
from fastapi import FastAPI
import joblib
import openai
import os
from dotenv import load_dotenv
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import nest_asyncio

# Apply async event loop patch for Jupyter
nest_asyncio.apply()

# Load API Key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Load trained model
model = joblib.load("query_classification_model.pkl")

# Load dataset to recreate TF-IDF vectorizer
df = pd.read_csv("tacticone_dataset.csv")
vectorizer = TfidfVectorizer()
vectorizer.fit(df["processed_query"])  # Fit on full dataset

# Initialize FastAPI
app = FastAPI()

# OpenAI Client
client = openai.OpenAI(api_key=openai_api_key)

# Response dictionary for rule-based responses
responses = {
    "order_status": "You can track your order in the 'My Orders' section.",
    "returns": "You can return your item within 30 days. Visit our Returns page.",
    "product_information": "You can find product details on the product page.",
    "technical_support": "For technical issues, please visit our Help Center or contact support.",
}

# AI-powered response function
def generate_ai_response(query):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful support assistant."},
            {"role": "user", "content": query}
        ]
    )
    return response.choices[0].message.content

# Query classification function
def classify_and_respond(query):
    processed_query = query.lower()
    query_tfidf = vectorizer.transform([processed_query])
    predicted_category = model.predict(query_tfidf)[0]
    
    # Get predefined response or fallback to AI response
    response = responses.get(predicted_category, generate_ai_response(query))
    
    return {"category": predicted_category, "response": response}

# Define API endpoint
@app.post("/chat/")
async def chat(query: str):
    return classify_and_respond(query)

# Start FastAPI in the background
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [50930]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [ ]:
!pip install fastapi uvicorn nest_asyncio joblib openai python-dotenv pandas scikit-learn

In [ ]:
import nest_asyncio
nest_asyncio.apply()


In [ ]:
import requests

# Define API URL
url = "http://127.0.0.1:8000/chat/"

# Send a test request
query = {"query": "Where is my order?"}
response = requests.post(url, params=query)

# Display response
print(response.json())
